In [1]:
from deepfix.client import DeepFixClient


d:\workspace\repos\deepfix\.venv\Lib\site-packages\deepchecks\core\serialization\dataframe\html.py:16: UserWarning:

pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.



In [2]:
client = DeepFixClient(api_url="http://localhost:8844",timeout=120)

# Image classification Dataset

### Ingestion

In [5]:
from deepfix.client.zoo.datasets.foodwaste import load_train_and_val_datasets

In [ ]:
# Load image datasets
train_data, val_data = load_train_and_val_datasets(
    image_size=448,
    batch_size=8,
    num_workers=4,
    pin_memory=False,)

In [ ]:
dataset_name="cafetaria-foodwaste-lstroetmann"

dataset_logging_pipeline = client.ingest_dataset(dataset_name=dataset_name,
                                                train_data=train_data,
                                                test_data=val_data,
                                                train_test_validation=True,
                                                data_integrity=True,
                                                batch_size=8,
                                                overwrite=False
                                                )

### Analyze Dataset

In [3]:
dataset_name="cafetaria-foodwaste-lstroetmann"

In [4]:
result = client.diagnose_dataset(dataset_name=dataset_name)

Output()

✓ Analysis complete!

In [5]:
print(result.to_text())

╭──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│                                               DEEPFIX ANALYSIS RESULT                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────── Summary ───────────────────────────────────────────────────────╮
│ The combined analysis reveals a critically flawed dataset with multiple compounding issues that render the current   │
│ machine learning approach invalid. The most severe problem is the catastrophic train-test split where 75% of test    │
│ labels are unseen in training, making evaluation meaningless. This is exacerbated by extremely small dataset size    │
│ (215 training samples) and significant visual domain mismatch across brightness, contrast, and color properties. The │
│ dataset appears to come from inconsistent collection sources or conditions. Immediate action is required to          │
│ completely re-split the data with proper stratification, significantly expand data collection, implement domain      │
│ adaptation techniques, and adopt robust cross-validation for reliable evaluation. The current setup cannot produce a │
│ trustworthy computer vision model.                                                                                   │
╰──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                      Summary Statistics                                      
 Metric                          Value                                                        
 Total Findings                  12                                                           
 Agents Involved                 DatasetArtifactsAnalyzer, DeepchecksArtifactsAnalyzer,       
                                 CrossArtifactReasoningAgent                                  
 Severity Distribution           HIGH: 9  MEDIUM: 3                                           

                                                HIGH Severity Issues (9)                                                
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Agent                         ┃ Finding                                ┃ Action                                  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ DatasetArtifactsAnalyzer      │ Extremely small training dataset size  │ Implement aggressive data augmentation  │
│   │                               │ Evidence: Training set contains only   │ and consider collecting more training   │
│   │                               │ 215 samples, which is insufficient for │ data                                    │
│   │                               │ reliable computer vision model         │ Small dataset size is the primary       │
│   │                               │ training                               │ driver of overfitting as models can     │
│   │                               │                                        │ easily memorize all training examples   │
│ 2 │ DatasetArtifactsAnalyzer      │ Dataset lacks diversity and            │ Implement comprehensive data            │
│   │                               │ augmentation strategy                  │ augmentation including rotations,       │
│   │                               │ Evidence: No augmentation operations   │ flips, color adjustments, and cropping  │
│   │                               │ specified and small dataset suggests   │ Augmentation artificially increases     │
│   │                               │ limited visual diversity               │ dataset diversity and helps prevent     │
│   │                               │                                        │ memorization of specific examples       │
│ 3 │ DeepchecksArtifactsAnalyzer   │ Severe train-test split contamination  │ Completely re-split the dataset using   │
│   │                               │ with 75% new labels in test set        │ proper stratified random sampling       │
│   │                               │ Evidence: New Labels check failed: 75% │ The current split is fundamentally      │
│   │                               │ of labels found in test set were not   │ flawed, making test set evaluation      │
│   │                               │ in train set. New labels most common   │ meaningless. A proper stratified split  │
│   │                               │ in test set: ['4', '1', '37']          │ ensures both sets have similar class    │
│   │                               │                                        │ distributions and no unseen classes     │
│ 4 │ DeepchecksArtifactsAnalyzer   │ Extreme label distribution drift       │ Implement stratified k-fold             │
│   │                               │ between train and test sets            │ cross-validation for reliable model     │
│   │                               │ Evidence: Label Drift check failed:    │ assessment                              │
│   │                               │ Cramer's V score of 0.92 for 'Samples  │ With such severe drift, single          │
│   │                               │ Per Class' indicates near-complete     │ train-test splits are unreliable.       │
│   │                               │ distribution mismatch between train    │ Cross-validation provides more robust   │
│   │                               │ and test sets                          │ performance estimates across multiple   │
│   │                               │                                        │ splits                                  │
│ 5 │ DeepchecksArtifactsAnalyzer   │ Significant image property drift       │ Apply robust data augmentation and      │
│   │                               │ affecting model generalization         │ domain adaptation techniques            │
│   │  

                                               MEDIUM Severity Issues (3)                                               
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Agent                         ┃ Finding                                ┃ Action                                  ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ DatasetArtifactsAnalyzer      │ Limited test set size affecting        │ Use cross-validation and consider       │
│   │                               │ evaluation reliability                 │ collecting more test data for reliable  │
│   │                               │ Evidence: Test set contains only 160   │ evaluation                              │
│   │                               │ samples, making performance metrics    │ Small test sets can lead to misleading  │
│   │                               │ potentially unreliable with high       │ performance estimates and hide          │
│   │                               │ variance                               │ overfitting issues                      │
│ 2 │ DatasetArtifactsAnalyzer      │ Potential distribution mismatch        │ Investigate data collection process and │
│   │                               │ between train and test sets            │ ensure proper random splitting of       │
│   │                               │ Evidence: Mean values differ between   │ train/test data                         │
│   │                               │ train (0.635) and test (0.684) in blue │ Distribution mismatches can cause the   │
│   │                               │ channel, and standard deviations show  │ model to perform well on training but   │
│   │                               │ variation across channels              │ poorly on real-world data               │
│ 3 │ DeepchecksArtifactsAnalyzer   │ Dataset collection methodology appears │ Review and standardize data collection  │
│   │                               │ inconsistent                           │ procedures                              │
│   │                               │ Evidence: Multiple drift failures      │ Ensuring consistent collection          │
│   │                               │ suggest train and test data may come   │ conditions across all data splits is    │
│   │                               │ from different sources, times, or      │ crucial for building reliable computer  │
│   │                               │ collection conditions                  │ vision models                           │
└───┴───────────────────────────────┴────────────────────────────────────────┴─────────────────────────────────────────┘

                                                Agent-Specific Analysis                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Agent                         ┃ Findings  ┃ Artifacts                      ┃ Summary                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ DatasetArtifactsAnalyzer      │     4     │ DatasetArtifacts               │ N/A                                     │
│ DeepchecksArtifactsAnalyzer   │     4     │ DeepchecksArtifacts            │ N/A                                     │
│ CrossArtifactReasoningAgent   │     4     │ DatasetArtifacts,              │ The combined analysis reveals a         │
│                               │           │ DeepchecksArtifacts            │ critically flawed dataset with multiple │
│                               │           │                                │ compounding issues that render the      │
│                               │           │                                │ current machine learning approach       │
│                               │           │                                │ invalid. The most severe problem is the │
│                               │           │                                │ catastrophic train-test split where 75% │
│                               │           │                                │ of test labels are unseen in training,  │
│                               │           │                                │ making evaluation meaningless. This is  │
│                               │           │                                │ exacerbated by extremely small dataset  │
│                               │           │                                │ size (215 training samples) and         │
│                               │           │                                │ significant visual domain mismatch      │
│                               │           │                                │ across brightness, contrast, and color  │
│                               │           │                                │ properties. The dataset appears to come │
│                               │           │                                │ from inconsistent collection sources or │
│                               │           │                                │ conditions. Immediate action is         │
│                               │           │                                │ required to completely re-split the     │
│                               │           │                                │ data with proper stratification,        │
│                               │           │                                │ significantly expand data collection,   │
│                               │           │                                │ implement domain adaptation techniques, │
│                               │           │                                │ and adopt robust cross-validation for   │
│                               │           │                                │ reliable evaluation. The current setup  │
│                               │           │                                │ cannot produce a trustworthy computer   │
│                               │           │                                │ vision model.                           │
└───────────────────────────────┴───────────┴────────────────────────────────┴─────────────────────────────────────────┘